In [1]:
! pip3 install --upgrade google-cloud-aiplatform

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
Requirement already up-to-date: google-cloud-aiplatform in ./.venv/lib/python3.8/site-packages (1.40.0)


In [4]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
    HarmBlockThreshold,
    HarmCategory,
)



In [5]:
gemini_model = GenerativeModel("gemini-pro")

generation_config = GenerationConfig(
    temperature=0.1,
    max_output_tokens=2048,
)

safety_config = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}


In [6]:

def analyze_sentiment(voc):
  prompt = """You are a CPG marketer, You received the customer feedback as the below, analyze the sentiment with score (from 0 to 100) and feelings - ex. curious,  happy, sad, angry, etc.
customer feedback : {voc}
output : sentiment score, feelings
"""
  response = gemini_model.generate_content(prompt.format(voc=voc),generation_config=generation_config, safety_settings=safety_config, stream=False)
  return response



In [7]:
response = analyze_sentiment("여행이 즐거웠나?")

In [11]:
print(response.candidates[0].content)

role: "model"
parts {
  text: "Sentiment score: 80\nFeelings: Curious"
}



In [12]:
## Try more examples
print(analyze_sentiment("여행은 어뗘?").candidates[0].content)

role: "model"
parts {
  text: "Sentiment score: 70\nFeelings: Curious"
}



OK, gemini model can detect the emotional(feeling) category from the VoC.

Vice versa, we will try to make a phrase with various emotinal conditions. 

In [16]:
def generate_sentence_with_emotion(business_goal, emotional_condition):
  prompt = """You are a CPG marketer, create a single question for your customers by combining the given business goal and the desired emotional state.

business goal : {business_goal}

emotional condition : {emotional_condition}
"""
  response = gemini_model.generate_content(prompt.format(business_goal=business_goal, emotional_condition=emotional_condition),generation_config=generation_config, safety_settings=safety_config, stream=False)
  return response

In [22]:
business_goal = "금번 오사카 여행에 대한 감상평을 듣고 싶음"
emotional_condition = "친근하며 격의없이 아양 떠는 느낌으로"

response = generate_sentence_with_emotion(business_goal, emotional_condition)
print(response.candidates[0].content)

role: "model"
parts {
  text: "오사카에 다녀오셨는데, 가장 맘에 들었던 것이 무엇이었는지 알려주실래요?😊"
}



It's very formal response. Try to make another response.

In [20]:
generation_config = GenerationConfig(
    temperature=0.8,
    max_output_tokens=2048,
)

safety_config = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

In [30]:
emotional_condition = "친구 사이의 짧은 문장과 호기심을 담아서"

response = generate_sentence_with_emotion(business_goal, emotional_condition)
print(response.candidates[0].content)

role: "model"
parts {
  text: "오사카에 간 친구들에게 물어보고 싶은 말 한 마디가 있다면?"
}



In [35]:
def generate_sentence_with_emotion(business_goal, emotional_condition, generation_config, safety_config):
  prompt = """Create a single question for your customers by combining the given business goal and the desired tone of voice.

business goal : {business_goal}

tone of voice : {emotional_condition}
"""
  response = gemini_model.generate_content(prompt.format(business_goal=business_goal, emotional_condition=emotional_condition),generation_config=generation_config, safety_settings=safety_config, stream=False)
  return response

In [36]:
emotional_condition = "친구 사이의 짧은 문장과 호기심을 담아서"

response = generate_sentence_with_emotion(business_goal, emotional_condition, generation_config, safety_config)
print(response.candidates[0].content)

role: "model"
parts {
  text: "오사카 가볼 때 꼭 가봐야 할 곳이 궁금해!"
}



In [37]:
emotional_condition = "40대의 중년 남성 느낌으로 짧게"

response = generate_sentence_with_emotion(business_goal, emotional_condition, generation_config, safety_config)
print(response.candidates[0].content)

role: "model"
parts {
  text: "오사카 여행을 다녀오신 분들의 감상을 들려주세요."
}



In [38]:
emotional_condition = "10대의 여성 쳔구 느낌으로 짧게"

response = generate_sentence_with_emotion(business_goal, emotional_condition, generation_config, safety_config)
print(response.candidates[0].content)

role: "model"
parts {
  text: "오사카 갔다온 것 생각나~? 핵뿌듯한 것들 소개해줘ㅓ~ 뭐가 제일 쩝이냐?"
}

